In [46]:
import mysql.connector
import pandas as pd

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    user='root',
    password='Sursurpatil@4',
    host='127.0.0.1',
    database='user_analytics'
)

# Create a cursor object
cursor = conn.cursor()


In [47]:
DATA_PATH = r'E:\Project5\telcom_data p5.csv'
data = pd.read_csv(DATA_PATH)

In [48]:
import pickle

# Load the saved data from Task 4
with open('task4_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [49]:
for index, row in data.head(10).iterrows():
 print(f"Engagement: {row['engagement_score']}, Experience: {row['experience_score']}, Satisfaction: {row['satisfaction_score']}")

Engagement: 7528.192032213845, Experience: 1.0, Satisfaction: 843408406.6179165
Engagement: 865.5280196435468, Experience: 1.0, Satisfaction: 126037214.51034226
Engagement: 4415.580488852996, Experience: 1.0, Satisfaction: 558045719.8281446
Engagement: 8467.073619631901, Experience: 1.0, Satisfaction: 402506796.99645627
Engagement: 39279.07891332471, Experience: 1.0, Satisfaction: 1366386971.0266485
Engagement: 2422.2771642196526, Experience: 1.0, Satisfaction: 558441641.8517305
Engagement: 5100.678295419134, Experience: 1.0, Satisfaction: 562311119.5873492
Engagement: 3850.280176853899, Experience: 1.0, Satisfaction: 296604270.19574654
Engagement: 1997.4343234443274, Experience: 1.0, Satisfaction: 901430037.6902843
Engagement: 5866.562232687779, Experience: 1.0, Satisfaction: 686988638.9371362


In [50]:
for index, row in data.iterrows():
    sql = "INSERT INTO user_data (engagement_score, experience_score, satisfaction_score) VALUES (%s, %s, %s)"
    val = (float(row['engagement_score']), float(row['experience_score']), float(row['satisfaction_score']))
    cursor.execute(sql, val)

conn.commit()
cursor.close()
conn.close()